In [1]:
### Imports ###
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from astropy.io import fits
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
import os
from scipy.optimize import curve_fit
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
from GalPop import *
from tqdm.notebook import tqdm

In [2]:
### SET DATA LIMITS ###
IRAC_lim = 26
ra_range = (149.6, 150.52)  
dec_range = (1.74, 2.73)
z_range = (2., 3.)

# Assign Peaks to C20p

In [3]:
### LOAD DATA ###
# Cosmos data
cosmos = fits.open(r"C:\Users\sikor\OneDrive\Desktop\BigData\COSMOS2020\COSMOS2020_CLASSIC_R1_v2.0.fits")
c20p = cosmos[1].data

# MC iterations
MCp = np.load(r"C:\Users\sikor\OneDrive\Desktop\BigData\COSMOS2020\C20_MC_250_0_redo.npy")

## Interpolated sigma cube
sig_cube = fits.open(r"C:\Users\sikor\OneDrive\Desktop\benpy\ThirdTry\Hyperion_interpolated_cube_sigma.fits")

## Path for folders with .dat sigma files
sigPath = r"C:\Users\sikor\OneDrive\Desktop\benpy\HyperionPeaks"

print(f"Cosmos catalog = {np.shape(c20p)}")
print(f"MCs = {np.shape(MCp)}")

Cosmos catalog = (1720700,)
MCs = (1720700,)


In [4]:
## CUT DATA TO POTENTIALLY USABLE ##
g_idxs = np.where((c20p["ALPHA_J2000"] >= ra_range[0]) & (c20p["ALPHA_J2000"] <= ra_range[1])       # RA
                & (c20p["DELTA_J2000"] >= dec_range[0]) & (c20p["DELTA_J2000"] <= dec_range[1])     # DEC
                & ((c20p["IRAC_CH1_MAG"] <= IRAC_lim) | (c20p["IRAC_CH2_MAG"] <= IRAC_lim)) # IRAC
                & ((c20p["lp_type"] == 0) | (c20p["lp_type"] == 2)))        # LePhare type

c20p = c20p[g_idxs] # cosmos photometry
MCp = MCp[g_idxs]   # MC iterations

print(f"Potentially good galaxies = {np.shape(MCp)}")

Potentially good galaxies = (367677,)


In [5]:
## PEAKS FOR ORIGINAL SAMPLE ##

sigs = [5.0, 4.0, 3.0, 2.0]
sig_path = sigPath
sig_sums = [f"\peaks_sig{s}.dat" for s in sigs]
sig_folders = [f"\sig{s}_peaks" for s in sigs]

# Create the galaxy coordinates for this iteration
gal_coords = np.c_[c20p["ALPHA_J2000"], c20p["DELTA_J2000"], MCp[f"lp_zPDF"]]

# Find good galaxies for this pop
g_idxs = np.where((gal_coords[:,2] >= z_range[0]) & (gal_coords[:,2] <= z_range[1]))


# Create an instance
gp = GalPop(c20p["ID"][g_idxs], gal_coords[g_idxs], c20p["lp_mass_med"][g_idxs], verbose=False)

# Assign the peaks
gp.assignPeaks(sigs, sig_cube, sig_path, sig_sums, sig_folders, overwrite=True)

# Save file
gp.saveFile(f"GalPops_phot/c20p.npy")

In [6]:
## Create GalPop Objects ##
mc_iter = 10     # Number of differnet redshifts to use
sigs = [5.0, 4.0, 3.0, 2.0]
sig_path = sigPath
sig_sums = [f"\peaks_sig{s}.dat" for s in sigs]
sig_folders = [f"\sig{s}_peaks" for s in sigs]


for m in tqdm(range(mc_iter)):

    # Create the galaxy coordinates for this iteration
    gal_coords = np.c_[c20p["ALPHA_J2000"], c20p["DELTA_J2000"], MCp[f"MC_iter{m}"]]

    # Find good galaxies for this pop
    g_idxs = np.where((gal_coords[:,2] >= z_range[0]) & (gal_coords[:,2] <= z_range[1]))


    # Create an instance
    gp = GalPop(c20p["ID"][g_idxs], gal_coords[g_idxs], c20p["lp_mass_med"][g_idxs], verbose=False)

    # Assign the peaks
    gp.assignPeaks(sigs, sig_cube, sig_path, sig_sums, sig_folders, overwrite=True)

    # Save file
    gp.saveFile(f"GalPops_phot/MC_{m}.npy")

  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
cosmos = fits.open(r"C:\Users\sikor\OneDrive\Desktop\BigData\COSMOS2020\COSMOS2020_CLASSIC_R1_v2.0.fits")
c20p = cosmos[1].data
MCs = np.load(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/C20spec_MC_1000.npy", allow_pickle=True)

c20_sgals = np.take(c20p, MCs["ID"]-1, axis=0)

In [8]:
g_idxs = np.where((c20_sgals["ALPHA_J2000"] >= ra_range[0]) & (c20_sgals["ALPHA_J2000"] <= ra_range[1])       # RA
                & (c20_sgals["DELTA_J2000"] >= dec_range[0]) & (c20_sgals["DELTA_J2000"] <= dec_range[1])     # DEC
                & ((c20_sgals["IRAC_CH1_MAG"] <= IRAC_lim) | (c20_sgals["IRAC_CH2_MAG"] <= IRAC_lim))) # IRAC


c20_sgals = c20_sgals[g_idxs] # cosmos photometry
MCs = MCs[g_idxs]   # MC iterations

print(MCs.shape)

(40298,)


In [9]:
# Create the galaxy coordinates for this iteration
for sample in ["zs", "zg"]:

    gal_coords = np.c_[c20_sgals["ALPHA_J2000"], c20_sgals["DELTA_J2000"], MCs[sample]]

    # Find good galaxies for this pop
    g_idxs = np.where((gal_coords[:,2] >= z_range[0]) & (gal_coords[:,2] <= z_range[1]))


    # Create an instance
    gp = GalPop(c20_sgals["ID"][g_idxs], gal_coords[g_idxs], c20_sgals["lp_mass_med"][g_idxs], verbose=True)

    # Assign the peaks
    gp.assignPeaks(sigs, sig_cube, sig_path, sig_sums, sig_folders, overwrite=True)

    # Save file
    gp.saveFile(f"GalPops_spec/{sample}.npy")

Updating voxel assignments (self.voxels)
Updating overdensity values (self.ODs)
Updating n_sigmas values (self.n_sigmas)
Finding peaks for sigma = 5.0
Finding peaks for sigma = 4.0
Finding peaks for sigma = 3.0
Finding peaks for sigma = 2.0
Saved GalPop object to GalPops_spec/zs.npy, excluding []
Updating voxel assignments (self.voxels)
Updating overdensity values (self.ODs)
Updating n_sigmas values (self.n_sigmas)
Finding peaks for sigma = 5.0
Finding peaks for sigma = 4.0
Finding peaks for sigma = 3.0
Finding peaks for sigma = 2.0
Saved GalPop object to GalPops_spec/zg.npy, excluding []


In [10]:
## Create GalPop Objects ##
mc_iter = 10     # Number of differnet redshifts to use
sigs = [5.0, 4.0, 3.0, 2.0]
sig_path = sigPath
sig_sums = [f"\peaks_sig{s}.dat" for s in sigs]
sig_folders = [f"\sig{s}_peaks" for s in sigs]


# Create the galaxy coordinates for this iteration
for m in range(mc_iter):


    gal_coords = np.c_[c20_sgals["ALPHA_J2000"], c20_sgals["DELTA_J2000"], MCs[f"MC_iter{m}"]]

    # Find good galaxies for this pop
    g_idxs = np.where((gal_coords[:,2] >= z_range[0]) & (gal_coords[:,2] <= z_range[1]))


    # Create an instance
    gp = GalPop(c20_sgals["ID"][g_idxs], gal_coords[g_idxs], c20_sgals["lp_mass_med"][g_idxs], verbose=False)

    # Assign the peaks
    gp.assignPeaks(sigs, sig_cube, sig_path, sig_sums, sig_folders, overwrite=True)

    # Save file
    gp.saveFile(f"GalPops_spec/MC_{m}.npy")

## Combine Populations

In [11]:
ra_range = (149.6, 150.52)  
dec_range = (1.74, 2.73)
IRAC_cut = 25.

In [12]:
### PREP ###
sig_cube = fits.open(r"C:\Users\sikor\OneDrive\Desktop\benpy\ThirdTry\Hyperion_interpolated_cube_sigma.fits")

sigs = [2.0, 3.0, 4.0]
p_sums = []
for s in sigs:
    p_sums.append(np.genfromtxt(f"HyperionPeaks_new/peaks_sig{s}.dat", dtype=float))


cosmos_file = fits.open(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/COSMOS2020_CLASSIC_R1_v2.0.fits")
c20p = cosmos_file[1].data

In [16]:
### MAKE GAL POPS ###

m_iter = 250            # Max iterations
tot_pops = []           # Store total gal pops 


for m in tqdm(range(m_iter)):
    # Load Photo
    new_p = GalPop.loadFile(f"GalPops_phot/MC_{m}.npy")


    # Load specs
    new_s = GalPop.loadFile(f"GalPops_spec/MC_{m}.npy")



    # Cut repeat gals from photos
    pids = new_s.IDs   # IDs for the spectra
    new_p.del_objs(pids)       # Delete the objects

    # Combine
    new_tot = new_s.combine(new_p)
    
    # Remove based on IRAC cut
    c_idxs = new_tot.IDs.astype(int) - 1     # Get the indices of the cosmos ids in the new pop
    gals = c20p[c_idxs]     # pull these galaxies from cosmos
    
    # Find the objects which don't meet the specified criteria
    b_ids = np.where((gals["ALPHA_J2000"] <= ra_range[0]) | (gals["ALPHA_J2000"] >= ra_range[1]) # RA
                | (gals["DELTA_J2000"] <= dec_range[0]) | (gals["DELTA_J2000"] >= dec_range[1]) # Dec
            | ((gals["IRAC_CH1_MAG"] > IRAC_cut) & (gals["IRAC_CH2_MAG"] > IRAC_cut))   )       # IRAC cut


    new_tot.del_objs(new_tot.IDs[b_ids])

    tot_pops.append(new_tot)

  0%|          | 0/250 [00:00<?, ?it/s]

IndexError: index 5028 is out of bounds for axis 0 with size 5027

## Create subpopulations

In [17]:
### TOT-Z ###
pk_path = "HyperionPeaks_new/"
pk_sums = [f"peaks_sig{s}.dat" for s in sigs]

for idx, t in tqdm(enumerate(tot_pops)):

    t.subPop("field1", [-99, 4.0], 13. , [2.2, 2.3], pk_path, pk_sums[2], sig_cube, cosmo = cosmo, plot=f"popPlots/tot_{idx}")
    t.subPop("field2", [-99, 4.0], 13. , [2.65, 2.75], pk_path, pk_sums[2], sig_cube, cosmo = cosmo, plot=f"popPlots/tot_{idx}")

    for si, sigma in enumerate(sigs):
        t.subPop(f"o{sigma}", [sigma, np.inf], 13., [2.45, 2.55], pk_path, pk_sums[si], sig_cube, plot=f"popPlots/tot_{idx}")

    t.saveFile(f"GalPops_tot/MC_{idx}.npy")

0it [00:00, ?it/s]